In [ ]:
import numpy as np

def hamming_distance(str1, str2):
    return sum(c1 != c2 for c1, c2 in zip(str1, str2))
values = ['000', '001', '010', '011', '100', '101', '110', '111']

num_values = len(values)
hamming_distances = np.zeros((num_values, num_values), dtype=int)

for i in range(num_values):
    for j in range(num_values):
        hamming_distances[i, j] = hamming_distance(values[i], values[j])
print(hamming_distances)
def total_distance(P_XY):
 D=0
 for i, x in enumerate(values):
    for j, y in enumerate(values):
        d = hamming_distance(x, y)  
        D += P_XY[i][j] * d  
 return D


[[0 1 1 2 1 2 2 3]
 [1 0 2 1 2 1 3 2]
 [1 2 0 1 2 3 1 2]
 [2 1 1 0 3 2 2 1]
 [1 2 2 3 0 1 1 2]
 [2 1 3 2 1 0 2 1]
 [2 3 1 2 1 2 0 1]
 [3 2 2 1 2 1 1 0]]


In [ ]:
def compute_joint_distribution(P_X, P_Y):
    return np.outer(P_X, P_Y)

def compute_marginal(P_XY):
    P_X_marginal = np.sum(P_XY, axis=1)  
    P_Y_marginal = np.sum(P_XY, axis=0) 
    return P_X_marginal, P_Y_marginal

def compute_mutual_information(P_XY):
    
    P_X_marginal, P_Y_marginal = compute_marginal(P_XY)
    I_XY = 0
    for i in range(len(P_X_marginal)):
        for j in range(len(P_Y_marginal)):
                I_XY += P_XY[i, j] * np.log(P_XY[i, j] / (P_X_marginal[i] * P_Y_marginal[j]))
    
    return I_XY

In [86]:
def compute_next_joint_distribution(P_X,P_Y,d,theta):
    P_XY=np.zeros((len(P_X),len(P_X)))
    for i in range(len(P_X)):
        for j in range(len(P_Y)):
            temp1,temp2=0,0
            for k in range(len(P_Y)):
                temp2+=P_Y[k]*np.exp(theta*d[i][k])
            P_XY[i][j]=P_X[i]*P_Y[j]*np.exp(theta*d[i][j])/temp2
    return P_XY



In [ ]:

def compute_loss(P_X, P_Y_t, d, theta,D):
    total = 0
    for i in range(len(P_X)):
        inner_sum = 0
        for j in range(len(P_Y_t)):
            inner_sum += P_Y_t[j] * np.exp(theta * d[i, j]) 
        total += P_X[i] * np.log(inner_sum)  
    return total-theta*D


def compute_gradient(P_X, P_Y_t, d, theta,D):
    gradient = 0
    for i in range(len(P_X)):
        numerator = 0  #
        denominator = 0  
        
        for j in range(len(P_Y_t)):
            exp_term = np.exp(theta * d[i, j])  
            numerator += P_Y_t[j] * d[i, j] * exp_term
            denominator += P_Y_t[j] * exp_term
        
        gradient += P_X[i] * (numerator / denominator)  
    return gradient-D

def gradient_descent(P_X, P_Y_t, d, D,initial_theta, learning_rate, max_iter=1000, tolerance=1e-5):
    theta = initial_theta 
    for iteration in range(max_iter):
        gradient = compute_gradient(P_X, P_Y_t, d, theta,D)
        
        if np.abs(gradient) < tolerance:
            #print(f"Converged after {iteration + 1} iterations.")
            break
        
    
        theta -= learning_rate * gradient  # 梯度下降
        

        if iteration % 100 == 0:
            loss = compute_loss(P_X, P_Y_t, d, theta,D)
            #print(f"Iteration {iteration + 1}: θ = {theta:.5f}, Loss = {loss:.5f}, Gradient = {gradient:.5f}")
    
    return theta

P_X = np.array([0.1, 0.1, 0.1, 0.1,0.1,0.1,0.2,0.2]) 
P_Y = np.array([0.1, 0.3, 0.3,0.05,0.05,0.1,0.05,0.05])  
P_X_Y=compute_joint_distribution(P_X,P_Y)
D=total_distance(P_X_Y)
#print(compute_next_joint_distribution(P_X,P_Y,hamming_distances,1))
convergence=False
ite=0
while (convergence!=True):
 ite+=1
 #I_XY=compute_mutual_information(P_X_Y)
 #D=total_distance(P_X_Y)
#print(D)
 initial_theta = 0.5
 learning_rate = 0.0001
 max_iter = 1000
 tolerance = 1e-5

 #print(P_X_Y)
 optimal_theta = gradient_descent(P_X, P_Y, hamming_distances,D, initial_theta, learning_rate, max_iter, tolerance)
 #print(optimal_theta)
 P_X_Y=compute_next_joint_distribution(P_X,P_Y,hamming_distances,optimal_theta)
 #print(P_X_Y)
 #break
 P_Y_B=P_Y
 P_X,P_Y=compute_marginal(P_X_Y)
 I=compute_mutual_information(P_X_Y)
 print("MI: "+str(I))
 change=np.mean(abs(P_Y_B-P_Y))
 if change<=1e-4:
     convergence=True
     print(ite)
     print(P_Y)
     #print(I)
     #print(compute_mutual_information(P_X_Y))
 #print(f"Optimal θ: {optimal_theta:.5f}")


MI: 0.0703540485532546
MI: 0.0697847666727226
MI: 0.06916791205945101
MI: 0.06851827366076269
MI: 0.06784818889325296
MI: 0.06716776984004007
MI: 0.06648516411218643
MI: 0.06580682237639056
MI: 0.06513775494830684
MI: 0.06448176743033371
MI: 0.063841670613218
MI: 0.06321946329651425
MI: 0.06261648879938514
MI: 0.06203356712806358
MI: 0.06147110534407702
MI: 0.06092918886031754
MI: 0.06040765633606917
MI: 0.059906160652776576
MI: 0.059424218198579876
MI: 0.05896124841471964
MI: 0.05851660528666133
MI: 0.0580896022113496
MI: 0.057679531446230795
MI: 0.057285679147748465
MI: 0.056907336836471666
MI: 0.05654380998088375
MI: 0.056194424269533666
MI: 0.05585853003889156
MI: 0.05553550523904492
MI: 0.05522475724877148
MI: 0.054925723793218785
MI: 0.054637873169390325
MI: 0.05436070394516961
MI: 0.05409374426525139
MI: 0.0538365508708275
MI: 0.053588707918222435
MI: 0.053349825663988285
MI: 0.05311953906957133
MI: 0.05289750636695973
MI: 0.05268340761723208
MI: 0.052476943286253346
MI: 0.05227